In [ ]:
#for installing all the libraries
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install transformers
!pip install simpletransformers
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git


In [ ]:
#for checking gpu details if present any on machine
!nvidia-smi

In [ ]:
#for loading evaluate function
!pip install evaluate

In [ ]:
#for importing necessary all the libraries
import json
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from tqdm import tqdm
import torch
import nltk
import evaluate

In [5]:
# to load .jsonl dataset files in the code and returning dataframe of important columns for generation task.(mainly two columns, content and their respective spoilers)
def load_dataset(file_name):
    data = []
    with open(file_name, encoding='utf8') as f:
        for line in f:
            example = json.loads(line)
            post_text = example['postText'][0]
            title = example['targetTitle']
            paragraphs = ' '.join(example['targetParagraphs'])
            spoiler = example['spoiler'][0]
            data.append({'text': post_text + ' - ' + title + paragraphs, 'spoiler': spoiler})
    return pd.DataFrame(data)
train_data = load_dataset(r'Your train dataset jsonl file location')
validation_data = load_dataset(r'Your validation dataset jsonl file location')


In [7]:
# converting both dataframes to numpy to further apply preprocessing on it.
train_data = np.array(train_data)
validation_data = np.array(validation_data)

In [8]:
# For removing all the extra special characters from the data
characters = ['!','"','#','$','%','&','(',')','*','+','/',':',';','<','=','>','@','^','`','|','~','\t','[',']','{','}','\\','.','-']
for i in range(len(train_data)):
    for j in characters:
        train_data[i][0] = train_data[i][0].replace(j,"")
        train_data[i][1] = train_data[i][1].replace(j,"")

for i in range(len(validation_data)):
    for j in characters:
        validation_data[i][0] = validation_data[i][0].replace(j,"")
        validation_data[i][1] = validation_data[i][1].replace(j,"")

        
print(len(train_data))
print(len(validation_data))

train_data = pd.DataFrame(train_data)
validation_data = pd.DataFrame(validation_data)

3200
400


In [9]:
# Converting dataframes and storing them as .csv file
train_data.to_csv('Your train dataset jsonl file location which will store dataframe .csv format', index=False)
validation_data.to_csv('Your validation dataset jsonl file location which will store dataframe .csv format', index=False)

In [ ]:
# loading dataset in the form which is acceptable to pretrained model
from datasets import load_dataset

dataset_file = 'Your .csv file location from above'
dataset_file_1 = 'Your .csv file location from above'
train_data = load_dataset('csv', data_files=dataset_file)
val_data = load_dataset('csv', data_files=dataset_file_1)


In [11]:
print(train_data['train'][1]['1'])

2070


In [ ]:
# loading pretrained model t5-base on gpu of the server along with it's tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model_ckpt = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
# Creating embeddings of the data
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['0'] , max_length = 1024, truncation = True , padding= "max_length")
    
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['1'], max_length = 50, truncation = True , padding= "max_length")
        
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }
    
train = train_data.map(convert_examples_to_features, batched = True)
val = val_data.map(convert_examples_to_features, batched = True)
          

In [15]:
print(val_data)

DatasetDict({
    train: Dataset({
        features: ['0', '1'],
        num_rows: 400
    })
})


In [16]:
#Intializing data collector
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [17]:
#Defining training arguments
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

trainer_args = Seq2SeqTrainingArguments(
    output_dir='location where ypu want to store your model', num_train_epochs=10, warmup_steps=0,
    per_device_train_batch_size=2, per_device_eval_batch_size=2,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='epoch', save_steps=1e6,
    gradient_accumulation_steps=16
) 

In [18]:
# Passing trainer arguments
trainer = Seq2SeqTrainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=train['train'], 
                  eval_dataset=val['train'])

In [ ]:
# for starting training of the model
trainer.train()


In [22]:
# For saving model after training
model_pegasus.save_pretrained("location where ypu want to store your model")

In [23]:
#For saving it's tokenizer after saving
tokenizer.save_pretrained("location where ypu want to store your model tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')